# HOMEWORK 3

## 1) DATA COLLECTION

### 1.1 Get the list of animes
We start from the list of animes to include in your corpus of documents. In particular, we focus on the top animes ever list. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

The output of this step is a .txt file whose single line corresponds to an anime's url.

In [2]:
#import libraries
import requests
url = 'https://myanimelist.net/topanime.php'
response= requests.get(url)
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
#soup.find_all('a') #Finds all the links
#soup.find_all('tr') # Finds all the tables

In [4]:
def get_links_from_soup(soup):
    anime = []
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')))
    return anime

In [ ]:
tot_list= []
for lim in range(0, 20000, 50):
    if lim==0:
        new_url = url
    else:
        new_url = url+'?limit='+str(lim)
    response = requests.get(new_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tot_list += get_links_from_soup(soup)

In [ ]:
len(tot_list)

19122

In [ ]:
with open('out.txt', 'w') as f:
    for n, link in tot_list:
        f.write(link+'\n')

### 1.2. Crawl animes
Once you get all the urls in the first 400 pages of the list, you:

* Download the html corresponding to each of the collected urls.
* After you collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the * * * stopping point. More details in Important (2).
* Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

In [ ]:
with open("out.txt") as file:
    #the line will be the url
    number=0
    
    for url in file:
        respone=requests.get(url)
        soup=BeautifulSoup(response.content,"html.parser")
        soup=str(soup)
        #save file as htlm
        name=('article_%d.html' % (number))
        html_file = open(name,"w")
        html_file.write(soup)
        html_file.close()
        number+=1
       

**Important**

Due to the large amount of pages you need to download, follow the next tips that will help you speeding up several time-consuming operations.

[Save time downloading files] You are asked to crawl a considerable number of pages, which will take plenty of time. To speed up the operation, we suggest you to work in parallel with your group's colleagues or even generate code that works in parallel with all the CPUs available in your computer. In particular, using the same code, each component of the group can be in charge of downloading a subset of pages (e.g., the first 100). PAY ATTENTION: Once obtained all the pages, merge your results into an unique dataset. In fact, the search engine must look up for results in the whole set of documents.

[Save your data] It is not nice to restart a crawling procedure, given its runtime. For this reason, it is extremely important that for every time you crawl a page, you must save it with the name article_i.html, where i corresponds to the number of articles you have already downloaded. In such way, if something goes bad, you can restart your crawling procedure from the i+1-th document.

### 1.3 Parse downloaded pages
At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

* **Anime Name** (to save as animeTitle): *String*
* **Anime Type** (to save as animeType): *String*
* **Number of episode** (to save as animeNumEpisode): *Integer*
* **Release and End Dates of anime** (to save as releaseDate and endDate): Convert both release and end date into *datetime format*.
* **Number of members** (to save as animeNumMembers): *Integer*
* **Score** (to save as animeScore): *Float*
* **Users** (to save as animeUsers): *Integer*
* **Rank** (to save as animeRank): *Integer*
* **Popularity** (to save as animePopularity): *Integer*
* **Synopsis** (to save as animeDescription): *String*
* **Related Anime** (to save as animeRelated): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. *List of strings*.
* **Characters** (to save as animeCharacters): *List of strings*.
* **Voices** (to save as animeVoices): *List of strings*
* **Staff** (to save as animeStaff): Include the staff name and their responsibility/task in a *list of lists*.

For each anime, you create an anime_i.tsv file of this structure:

animeTitle \t animeType \t  ... \t animeStaff

## 2. SEARCH ENGINE
Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.

First, you must pre-process all the information collected for each anime by:

* Removing stopwords
* Removing punctuation
* Stemming
* Anything else you think it's needed

For this purpose, you can use the nltk library.

In [1]:
import pandas as pd
import string
import json
from operator import itemgetter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

### 2.1. Conjunctive query
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

In [2]:
df = pd.read_table("../shared_stuff/tsv_files/total_pages.tsv",
                       delimiter = "\t",
                       header = "infer",
                       on_bad_lines = "skip") # fix line 16473

In [12]:
def str_to_list(s):
    """
    This functions returns a list
    with each of the characters of the input string
    
    Arguments
        s : string
        
    Returns
        (list)
    """
    
    return [char for char in s]

def has_digits(s):
    """
    This function checks whether a string
    contains any digits
    
    Arguments
        s : string
        
    Returns
        (bool) True / False
    """
    
    return len([char for char in s if char.isdigit()]) != 0

def bad_words():
    """
    This function creates a list with words
    that should be excluded from the vocabulary
    during preprocessing, including punctuation,
    stopwords et similia
    
    Arguments
        none
        
    Returns
        (list)
    """
    
    punct = str_to_list(string.punctuation)
    punct += ["...", "''", "``", '""']
    
    stops = stopwords.words("english")
    
    other_suffixes = ["'s", "n't"]
    
    return punct + stops + other_suffixes

def preprocess(text, stemmer):
    """
    This function preprocesses some text (a document)
    by isolating each word, excluding stopwords et similia,
    and finally stemming them
    
    Arguments
        text : (string)
        stemmer : stemmer object, e.g. SnowBallStemmer()
    
    Returns
        (list) preprocessed input text
    """
    
    text = str(text)
    
    tokens = word_tokenize(text)
        
    return [stemmer.stem(w) for w in tokens 
            if w not in bad_words() and not has_digits(w)]

In [108]:
stemmer = SnowballStemmer("english")

df['synopsis_clean'] = df.apply(lambda row: preprocess(row['synopsis'], stemmer), 
                                axis = 'columns')

### 2.1.1) Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary of this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
where document_i is the id of a document that contains the word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

In [23]:
def create_vocab(corpus):
    """
    This function creates a set of unique
    and preprocessed words from a corpus
    
    Arguments
        corpus : pandas df column or list-like
    
    Returns
        vocab  : dictionary with the words as keys
                 and a unique integer for each as values
    """
    
    vocab = set()
    
    for doc in corpus:
        vocab.update(set(doc))

    return {word:idx for idx, word in enumerate(vocab)}
    

def save_dict_to_file(dct, filename):
    """
    This function saves a dictionary 
    to an external JSON file
    
    Arguments
        dct       : dictionary
        filename  : name of the file
        
    Returns
        void
    """
        
    with open(filename, "w") as file:
        json.dump(dct, file)
        

def read_dict_from_file(filename):
    """
    This function reads a dictionary
    from an external JSON file
        
    Arguments
        filename : name of the file
    
    Returns
        dct : dictionary with the contents of 'filename'
    """

    with open(filename, "r") as file:
        dct = json.loads(file.read())

    return dct

In [115]:
# only execute this cell the first time or 
# when the preprocessing changes!

vocab = create_vocab(df['synopsis_clean'])

save_dict_to_file(vocab, "vocabulary.json")

In [116]:
vocab = read_dict_from_file("vocabulary.json")

In [24]:
def create_inv_idx(corpus, vocab): 
    """
    This functions creates an inverted index list
    given a corpus of documents and a vocabulary
    
    Arguments
        corpus  : pandas df column or list-like
        vocab   : dictionary of all the words in the corpus
    
    Returns
        inv_idx : dictionary with the words as referenced in 'vocab' as keys 
                  and the lists of the documents each word is in as values       
    """
    
    inv_idx = {}
    
    for idx, word in zip(vocab.values(), vocab.keys()):
        inv_idx[idx] = [doc_id for doc_id, doc in enumerate(corpus) if word in doc]
    
    return inv_idx

In [118]:
# only execute this cell the first time or 
# when the preprocessing/vocabulary change!

inv_idx = create_inv_idx(df['synopsis_clean'], vocab)

#save_dict_to_file(inv_idx, "inv_idx.json")

In [119]:
# The keys of the dict we get are str, not int like when we created it
# I don't think it's necessary, but should we parse them when we read the json?
inv_idx = read_dict_from_file("inv_idx.json")

### 2.1.2) Execute the query
Given a query, that you let the user enter:

saiyan race
the Search Engine is supposed to return a list of documents.

What documents do we want?
Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

animeTitle
animeDescription
Url
Example Output:

animeTitle	animeDescription	Url
Fullmetal Alchemist: Brotherhood	...	https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood
Gintama	...	https://myanimelist.net/anime/28977/Gintama%C2%B0
Shingeki no Kyojin Season 3 Part 2	...	https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2
If everything works well in this step, you can go to the next point, and make your Search Engine more complex and better in answering queries.

In [120]:
def parse_query(query, vocab):
    """
    This functions converts the list of words
    input by the user into the list of the IDs
    the words are saved as in the vocabulary
    
    Arguments
        query : list of words
        vocab : vocabulary of words with the words as keys
                and their IDs as values
    Returns
        list of the IDs of the words in the query
    """
    
    parsed_query = []
    
    for word in query:
        try:
            parsed_query.append(vocab[stemmer.stem(word)])
        except KeyError:
            print(f"The term '{word}' wasn't found anywhere!")
    
    return parsed_query


def get_results(query):
    """
    This functions finds the documents all the words
    in the query are in.
    
    It finds them in three steps:
    1. creates a list of docs each word is in from the inverted index
    2. converts that list into a set
    3. intersects all those sets into a single set
       
    Arguments
        query : list of words as parsed by 'parse_query'
        
    Returns
        set with the documents that contain all the words in the query
    """
    
    return set.intersection(*[set(inv_idx[str(q)]) for q in query])


def get_df_entries(df, results,
                   url_file = "../shared_stuff/url_list.txt"):
    """
    This function filters the dataset so it only shows
    the rows which match the results, and adds a new column
    with the URL for the anime of each row.
    
    Arguments
        df       : pandas dataframe
        results  : set with the row indices to be filtered out
        url_file : external file with the URLs for each of the rows in df
    
    Returns
        df : filtered pandas dataframe
    """
    
    if not results:
        print("No results!")
        return
    
    with open(url_file, 'r') as file:
        url_list = file.read().split("\n")

    df = df.iloc[[*results]]
    df = df[["title", "synopsis"]]
    df = df.rename(columns = {"title": "animeTitle", 
                              "synopsis": "animeDescription"})
    
    df['animeUrl'] = itemgetter(*results)(url_list)
    
    return df

In [124]:
query = parse_query(input().split(), vocab)

# if at least one word in the query
# is in the vocabulary
if query:
    
    results = get_results(query)

    df_entries = get_df_entries(df, results)

    if df_entries: 
        display(df_entries)

 one piece goku


No results!


### 2.2) Conjunctive query & Ranking score
For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

* Find all the documents that contains all the words in the query.
* Sort them by their similarity with the query.
* Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use é * Python libraries) for maintaining the top-k documents.

To solve this task, you will have to use the tfIdf score, and the Cosine similarity. The field to consider it is still the synopsis. Let's see how.

### 2.2.1) Inverted index

In [ ]:
# bozza

# def create_inv_idx(corpus, vocab):
#     
#     inv_idx = {}
#     
#     for doc_num, doc in enumerate(corpus):
#         cnt = Counter(doc)
#         for word, idx in zip(vocab.keys(), vocab.values()):
#             inv_idx[idx] = (cnt[word])
#     
#     return inv_idx

### 2.2.2) Execute the query

## 3. DEFINE A NEW SCORE!

Now it's your turn. Build a new metric to rank animes based on the queries of their users.

In this scenario, a single user can give in input more information than the single textual query, so you need to take into account all this information, and think a creative and logical way on how to answer at user's requests.

#### Practically:

The user will enter you a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.<br>
<newline>

Once you have the documents, you need to sort them according to your new score. In this step you won't have anymore to take into account just the plot of the documents, you must use the remaining variables in your dataset (or new possible variables that you can create from the existing ones...). You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.<br>
<newline>

**Q: How to sort them?** <br>
**A: Allow the user to specify more information** that you find in the documents, and define a new metric that ranks the results based on the new request. You can also use other information regarding the anime to score some animes above others.<br>
N.B.: You have to define a scoring function, not a filter!

The output, must contain:

* **animeTitle**
* **animeDescription**
* **Url**
* **The new similarity score of the documents** with respect to the query


Are the results you obtain better than with the previous scoring function. Explain and compare results


## Solution

The basic idea is that a user can search for the words in the synopsis in the 2.1 point, so here we want to give the possibility to the user to query the data in the points he preferes from:
* title
* staff
* characters
* voices
* synopsis

Given that the query shoudld be in the form:<br>
"```word1 word2 [where_to_search] word3 word4 [where_t_s2] ...```"<br>
and the program will return the documents that contains word1 AND word2 in the field 'where_to_search' AND contains word3 AND word4 in the field 'where_t_s2'
<newline>

Hence the idea is to create an inverted index for each of thie fields and then parse the query and obtain the document that match it.<br>
We need to sort this document accordingly with a customized score and return for each of them the neccessarly info

In [20]:
import os
def actual_indexes(path='../data/indexes'):
    ret = dict()
    for idx_dir in os.listdir(path):
        if idx_dir.startswith('.'): continue
        idx_path = os.path.join(path,idx_dir)
        ret[idx_dir] = idx_path
    return ret
actual_indexes()

{'synopsis': '../data/indexes/synopsis'}

In [7]:
import ast

def join_list_of_ch_vc(col_name, df):
    return df.apply(lambda row: ' '.join(ast.literal_eval(row[col_name])), 
                                axis = 'columns')

def join_list_of_staff(df):
    return df.apply(lambda row: ' '.join([el[0] for el in ast.literal_eval(row['staff'])]), 
                                axis = 'columns')

def preprocess_column(col_name, df):
    return df.apply(lambda row: (preprocess(row[col_name], SnowballStemmer('english'))), axis = 'columns')

In [10]:
import ast
df = df[df.isna().sum(axis=1)==0]

In [21]:
def preprocessing_staff(df):
    string_staff = df.apply(lambda row: ' '.join([el[0] for el in ast.literal_eval(row['staff'])]), 
                                axis = 'columns')
    print(f"Converted all the lists in strings, starting the preprocessing...")
    prepr_staff = df.apply(lambda row: (preprocess(row['staff'], SnowballStemmer('english'))), axis = 'columns')
    return prepr_staff

preprocessing_staff(df)


0        [cook, justin, produc, yonai, noritomo, produc...
1        [fujita, youichi, director, storyboard, plan, ...
2        [yabuta, shuuhei, produc, wada, jouji, produc,...
3        [iwasa, gaku, produc, yasuda, takeshi, produc,...
4        [ibata, yoshihid, director, aketagawa, jin, so...
                               ...                        
19112                                                   []
19113                          [ochiai, masamun, director]
19114                               [raika, ken, director]
19115                                                   []
19116    [araki, hideki, director, ateli, kaguya, origi...
Length: 19115, dtype: object

In [17]:
string_staff = df.apply(lambda row: ' '.join([el[0] for el in ast.literal_eval(row['staff'])]), 
                                axis = 'columns')
prepr_staff = df.apply(lambda row: (preprocess(row['staff'], SnowballStemmer('english'))), axis = 'columns')

string_voices = df.apply(lambda row: ' '.join(ast.literal_eval(row['voices'])), 
                                axis = 'columns')
prepr_voices = df.apply(lambda row: (preprocess(row['voices'], SnowballStemmer('english'))), axis = 'columns')

string_characters = df.apply(lambda row: ' '.join(ast.literal_eval(row['characters'])), 
                                axis = 'columns')
prepr_characters = df.apply(lambda row: (preprocess(row['characters'], SnowballStemmer('english'))), axis = 'columns')

prepr_title = df.apply(lambda row: (preprocess(row['title'], SnowballStemmer('english'))), axis = 'columns')
                                


In [18]:
voc_staff = create_vocab(prepr_staff)
voc_voices = create_vocab(prepr_voices)
voc_characters = create_vocab(prepr_characters)
voc_title = create_vocab(prepr_title)

In [25]:
idx_title = create_inv_idx(prepr_title, voc_title)

In [27]:
df.tail(10)

,title,type,episodes,start_date,end_date,score,users,ranked,popularity,members,synopsis,related_anime,characters,voices,staff
19107,Hi Gekiga Ukiyoe Senya Ichiya,Movie,1,1969-10-29 00:00:00,None,None,None,None,12667.0,556.0,An adult movie about a painter whose pictures ...,None,[],[],[]
19108,Himitsu no Kichi,OVA,2,2022-02-04 00:00:00,None,None,None,None,17702.0,105.0,No synopsis information has been added to this...,None,[],[],[]
19109,Ikenai Boy: Ikasu Maruhi Hand Power,OVA,1,1990-09-28 00:00:00,None,None,None,None,13353.0,431.0,Sequel of Ikenai Boy.,['Ikenai Boy'],[],[],[]
19110,Isaku: Tsumi to Batsu,Special,1,1999-03-26 00:00:00,None,None,None,None,15535.0,235.0,No synopsis information has been added to this...,['Isaku'],[],[],[]
19111,Kaihouku: Chikan Harem,OVA,1,2007-03-25 00:00:00,None,None,None,None,18237.0,79.0,No synopsis information has been added to this...,['Tsuukin Kairaku: Chikan de Go!!'],[],[],[]
19112,Konbini Shoujo Z,OVA,None,None,None,None,None,None,12776.0,560.0,No synopsis information has been added to this...,None,[],[],[]
19113,Korogashi Ryouta,OVA,3,1990-11-21 00:00:00,1991-11-21 00:00:00,None,None,None,14511.0,309.0,Takao Ryota is a pervy but well-meaning biker-...,['Korogashi Ryouta'],[],[],"[['Ochiai, Masamune', ['Director']]]"
19114,Kyonyuu Elf Oyako Saimin,OVA,2,2022-01-28 00:00:00,None,None,None,None,16931.0,151.0,No synopsis information has been added to this...,None,"['Agraliel, Ephildis', 'Agraliel, Almia']",[],"[['Raika, Ken', ['Director']]]"
19115,Mahou Shoujo Elena DVD-BOX Special,Special,1,2015-12-11 00:00:00,None,None,None,None,14501.0,330.0,Short special included in the DVD-BOX of,['Mahou Shoujo Elena'],[],[],[]
19116,Mama x Holic: Miwaku no Mama to Amaama Kankei ...,OVA,2,2021-11-26 00:00:00,None,None,None,None,11074.0,1025.0,No synopsis information has been added to this...,['Love x Holic: Miwaku no Otome to Hakudaku Ka...,[],[],"[['Araki, Hideki', ['Director']], ['Atelier Ka..."


In [38]:
import csv
df_dirty = pd.read_table("../shared_stuff/tsv_files/total_pages.tsv",
                       delimiter = "\t",
                       header = "infer",quoting=csv.QUOTE_NONE, error_bad_lines=False)


/Users/alessandro/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
tot_lines = open("../data/tsv_files/total_pages.tsv",'r').readlines()


In [39]:
df_dirty['title']

0                         Fullmetal Alchemist: Brotherhood
1                              Gintama° (Gintama Season 4)
2                       Shingeki no Kyojin Season 3 Part 2
3                                              Steins;Gate
4                                 Fruits Basket: The Final
                               ...                        
19118                                     Konbini Shoujo Z
19119                                     Korogashi Ryouta
19120                             Kyonyuu Elf Oyako Saimin
19121                   Mahou Shoujo Elena DVD-BOX Special
19122    Mama x Holic: Miwaku no Mama to Amaama Kankei ...
Name: title, Length: 19123, dtype: object

In [34]:
for l in tot_lines:
    title = l.split('\t')[0]
    print(f"The film: {title} is in dirty_df? {title in df_dirty['title']}")

The film: title is in dirty_df? False
The film: Fullmetal Alchemist: Brotherhood is in dirty_df? False
The film: Gintama° (Gintama Season 4) is in dirty_df? False
The film: Shingeki no Kyojin Season 3 Part 2 is in dirty_df? False
The film: Steins;Gate is in dirty_df? False
The film: Fruits Basket: The Final is in dirty_df? False
The film: Gintama' (Gintama Season 2) is in dirty_df? False
The film: Hunter x Hunter (2011) (Hunter x Hunter) is in dirty_df? False
The film: Ginga Eiyuu Densetsu (Legend of the Galactic Heroes) is in dirty_df? False
The film: Gintama': Enchousen (Gintama: Enchousen) is in dirty_df? False
The film: Gintama: The Final is in dirty_df? False
The film: Gintama. (Gintama Season 5) is in dirty_df? False
The film: 3-gatsu no Lion 2nd Season (March Comes In Like A Lion 2nd Season) is in dirty_df? False
The film: Koe no Katachi (A Silent Voice) is in dirty_df? False
The film: Clannad: After Story (Clannad ~After Story~) is in dirty_df? False
The film: Gintama is in dir

## 5. Algorithmic question
You consult for a personal trainer who has a back-to-back sequence of requests for appointments. A sequence of requests is of the form > 30, 40, 25, 50, 30, 20 where each number is the time that the person who makes the appointment wants to spend. You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, [30, 50, 20] is an acceptable solution (of duration 100), but [30, 40, 50, 20] is not, because 30 and 40 are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is [40, 50, 20], of total duration 110.

* Write an algorithm that computes the acceptable solution with the longest possible duration.
* Implement a program that given in input an instance in the form given above, gives the optimal solution.

In [ ]:
def algoritmo (array):
    a=0
    b=array[0]
    for elem in array[1:]:
        n=max(a+elem,b)
        a=b
        b=n
    return b

In [ ]:
array=[3,4,5,60,4]
algoritmo(array)

64